In [1]:
import pandas as pd

In [31]:
from tqdm import tqdm

In [2]:
jobs_df = pd.read_excel('de_jobs_with_clusters.xlsx')

In [14]:
titles_df = jobs_df[['job_id', 'cluster', 'descr_list']].copy()

In [15]:
titles_df

,job_id,cluster,descr_list
0,0053e5f78b135d5a,4,mit einem team von mitarbeiterinnen und mitar...
1,00ef1c15bf5b5939,1,location hamburg berlin kln dsseldorf gtersloh...
2,0176a1dc54841716,4,wir in der bi domain logistics management setz...
3,028876104a6b2d26,2,bochum unbefristet vollzeit it wir freuen u...
4,034b31000a95988c,1,in diesem video geben wir persnliche einblick...
...,...,...,...
940,https://de.indeed.com/viewjob?jk=fdd8d81f544bf511,2,wir verwenden cookies um inhalte und anzeigen ...
941,https://de.indeed.com/viewjob?jk=fe322edbe2225eb6,2,du suchst das besondere das spezielle das wowd...
942,https://de.indeed.com/viewjob?jk=fef83104f3acd912,4,job description in order to take our engineeri...
943,https://de.indeed.com/viewjob?jk=ff0d468c9afc7eb4,2,english version below die deutsche bank db kan...


In [16]:
titles_df['descr_list'][0]

'mit einem team von  mitarbeiterinnen und mitarbeitern arbeiten wir an digitalen consumer marketing projekten in den bereichen consulting content user experience  design und analytics und natrlich im front und backenddevelopment du magst herausfordernde projekte in interdisziplinrer zusammenarbeit du hast sowohl ein auge frs detail als auch den blick fr das groe ganze dann suchen wir genau dich kontaktiere uns damit wir uns kennen lernen knnen nicht nur digital sondern ganz persnlich wir freuen uns daraufwir verwenden cookies um inhalte und anzeigen zu personalisieren den traffic zu analysierennsowie um unsere dienstleistungen zu verbessern darber hinaus verwenden wir notwendigencookies um arbeitgeber darber zu informieren wie viele klicks ihre stellenanzeigen erhaltennhaben sie erkennen an dass ihre nutzung von indeed unseren richtlinien zur verwendung von cookies unterliegt und empfehlen ihnen diese zu lesen'

In [17]:
titles_df['words_count'] = titles_df['descr_list'].apply(lambda x: len(str(x).split()))

In [18]:
titles_df['words_count'].mean()

228.3820105820106

In [19]:
titles_df['words_count'].median()

174.0

# Lemmatizations

In [66]:
import spacy

In [67]:
nlp_en = spacy.load('en_core_web_sm')
nlp_de = spacy.load('de_core_news_sm')

In [230]:
# Assuming 'tokens' is your list of tokens
lemmatized_tokens = [nlp(token)[0].lemma_ for token in selected_tokens1]  # not bad

In [ ]:
nlp = spacy.load('de_core_news_sm')
lemmatized_tokens1 = [nlp(token)[0].lemma_ for token in lemmatized_tokens]

In [20]:
import numpy as np
from nltk.corpus import stopwords
from string import punctuation

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
german_stop_words = stopwords.words('german')

In [68]:
english_stop_words = stopwords.words('english')

In [87]:
tqdm.pandas()

In [22]:
gender_labels = ['mfx', 'mfd', 'wmd', 'mwd', 'fmd', 'fmx', 'dfm']

In [90]:
def token_processing(text):
    tokens = str(text).split()
    tokens = [token for token in tokens if token not in german_stop_words\
              and token != " "
              and token.strip() not in punctuation
              and token not in gender_labels
              and token not in english_stop_words
              ]
    # Assuming 'tokens' is your list of tokens
    tokens = [nlp_de(nlp_en(token)[0].lemma_)[0].lemma_ for token in tokens]
    tokens = list(set(tokens))

    text = " ".join(tokens)
    return text

In [91]:
titles_df['cleaned_text'] = titles_df['descr_list'].progress_apply(token_processing)

100%|██████████| 945/945 [09:31<00:00,  1.65it/s]


In [92]:
titles_df['cleaned_text'] = titles_df['cleaned_text'].str.lower()
titles_df['cleaned_text'] = titles_df['cleaned_text'].str.replace('[','').replace(']', '')
titles_df['cleaned_text']  = titles_df['cleaned_text'].str.replace('[^a-zA-Z ]', '', regex=True)  # replace all non Latin symbols

In [93]:
titles_df1 = titles_df[['cleaned_text', 'job_id']].copy()

In [94]:
titles_df1.dropna()

,cleaned_text,job_id
0,analytic kontaktiern arbeitgeber knn hinaus ve...,0053e5f78b135d5a
1,unabhngig datum groen tag verwenden deutschlan...,00ef1c15bf5b5939
2,bei gemeinsaman arbeitgeber excellence unterwe...,0176a1dc54841716
3,wohlfhlen uninitiativen gesamt sicherheit verw...,028876104a6b2d26
4,urlaub geburtsbeihilfe arbeitgeber tag verwend...,034b31000a95988c
...,...,...
940,arbeitgeber hinaus verwenden cookie informiere...,https://de.indeed.com/viewjob?jk=fdd8d81f544bf511
941,nachhaltigkeit softwareentwicklung sicherheit ...,https://de.indeed.com/viewjob?jk=fe322edbe2225eb6
942,writen worry act submit fidelity easy north cy...,https://de.indeed.com/viewjob?jk=fef83104f3acd912
943,beachten lear datum oral operation schrifen ge...,https://de.indeed.com/viewjob?jk=ff0d468c9afc7eb4


In [95]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(titles_df1['cleaned_text'])

In [96]:
# Convert the matrix into a DataFrame
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [97]:
tfidf_df.sum().sort_values()

usw                    0.061858
schriftlichen          0.061858
umsetzbare             0.061858
ausfhrungsstandard     0.061858
maschinellem           0.061858
                        ...    
notwendigencookie     45.962586
informieren           45.962586
indeedn               45.962586
unterliegen           45.962586
analysierennsowie     45.962586
Length: 14564, dtype: float64

In [52]:
import seaborn as sns

In [53]:
from matplotlib import pyplot as plt

In [54]:
tfidf_df.sum().to_frame()

,0
aa,0.033010
aachen,0.523654
aasap,0.061241
ab,3.336843
abaqus,0.038354
...,...
zwei,0.449456
zweimal,0.063131
zweites,0.131335
zwingend,0.237594


In [98]:
token_freq = pd.DataFrame(tfidf_df.sum(), columns=['freq'])

In [82]:
token_list = token_freq.reset_index()['index'].to_list()

In [84]:
tokens = [nlp_de(nlp_en(token)[0].lemma_)[0].lemma_ for token in tqdm(token_list)]

100%|██████████| 17521/17521 [01:21<00:00, 215.65it/s]


In [85]:
tokens = set(list(tokens))

In [86]:
len(tokens)

14747

In [99]:
q1 = token_freq['freq'].quantile(0.25)
q2 = token_freq['freq'].quantile(0.5)
q3 = token_freq['freq'].quantile(0.75)

# Print quantiles
print(f"25th percentile (Q1): {q1}")
print(f"50th percentile (Q2, Median): {q2}")
print(f"75th percentile (Q3): {q3}")

25th percentile (Q1): 0.10782296361261831
50th percentile (Q2, Median): 0.16574391238009817
75th percentile (Q3): 0.37879139055590877


In [118]:
token_freq['freq'].quantile(0.925)

1.5057778216755073

In [120]:
token_freq.loc[token_freq['freq']>token_freq['freq'].quantile(0.925)].to_excel('tfidf_tokens.xlsx')

In [105]:
token_freq1 = token_freq.reset_index()

In [107]:
token_freq1.loc[token_freq1['index']=='java']

,index,freq
6868,java,1.545396


# Selected Tokens

In [122]:
selected_tokens = pd.read_excel('tfidf_tokens.xlsx')

In [123]:
selected_tokens

,token,freq
0,analysierennsowie,45.962586
1,anzeigen,45.962586
2,cookie,45.962586
3,empfehlen,45.962586
4,erhaltennhaben,45.962586
...,...,...
1085,komman,1.512492
1086,bernehman,1.509167
1087,werden,1.507841
1088,leipzig,1.507704


In [126]:
tfidf_df1 = tfidf_df[selected_tokens['token'].to_list()].copy()

# Clusterization

In [128]:
from sklearn.cluster import KMeans

n_clusters = 5  # replace with the number of clusters you want

# Prepare the data for clustering
X = tfidf_df1

# Initialize the KMeans object
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# Perform clustering
tfidf_df1['cluster'] = kmeans.fit_predict(X)

/Users/dp_user/PycharmProjects/arbeite_parser/venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [130]:
tfidf_df1['job_id'] = titles_df['job_id']

In [134]:
jobs_df['cluster_descr'] = tfidf_df1['cluster']

In [136]:
jobs_df[['cluster_descr', 'job_id']].groupby('cluster_descr').count()

,job_id
cluster_descr,
0,148
1,53
2,11
3,299
4,434


In [137]:
jobs_df.to_excel('clustered_de_jobs.xlsx', index=False)

In [138]:
## data is too messed, need to clean better